# Set up

In [ ]:
!pip install albumentations
!pip install opencv-python
!pip install ultralytics
!pip install roboflow
!pip install inference supervision

In [5]:
import os, shutil
from os import listdir
from os.path import isfile, join
import cv2
import albumentations as A
import copy
import random
import numpy as np
from ultralytics import YOLO
import torch
import shutil

In [ ]:
# If you don't have yolov9 set up
!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9
!pip install -r requirements.txt -q

In [ ]:
# If you already have yolov9 in your repo
%cd yolov9

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="HV9jB1dcJZHZd1YO5S7C") 
project = rf.workspace("comp400").project("comp400-fish-detection")
dataset = project.version(3).download("yolov9")

# Train (training with on roboflow's website is easier - ignore this section)

In [ ]:
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-e.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
!python3 train.py \
     --data COMP400-Fish-detection-3/data.yaml \
     --weights weights/gelan-c.pt \
     --cfg models/detect/gelan-c.yaml \
     --hyp hyp.scratch-high.yaml \
     --device cpu
     

# Deploy

In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key='HV9jB1dcJZHZd1YO5S7C')
project = rf.workspace('comp400').project('comp400-fish-detection')
model = project.version(4).model

loading Roboflow workspace...
loading Roboflow project...


In [3]:
import os 

data_dir = "../data"
file_name = "geo.txt"

with open(os.path.join(data_dir, file_name), "r") as f:
     lines = f.readlines()
     lines = lines[1:]  # Skip the header line

filenames_raw, filenames_depth = [], []

for line in lines:
     filename_raw, filename_depth, _, _, _, _, _, _ = line.strip().split()
     filenames_raw.append(filename_raw)
     filenames_depth.append(filename_depth)


In [4]:
with open("prediction.txt", "w") as f:
     f.write("filename_raw filename_depth x y width height")
     for i in range(len(filenames_raw)):
          prediction = model.predict(os.path.join(data_dir, filenames_raw[i]), confidence=50, overlap=50).json()["predictions"]
          print(prediction)
          for j in range(len(prediction)):
               instance = prediction[j]
               x = instance["x"]
               y = instance["y"]
               width = instance["width"]
               height = instance["height"]
               f.write(f"\n{filenames_raw[i]} {filenames_depth[i]} {x} {y} {width} {height}")
     
     # uncoment next line if you want to visualize predictions
     # 

[{'x': 176.0, 'y': 180.5, 'width': 104.0, 'height': 53.0, 'confidence': 0.8466671109199524, 'class': 'fish1', 'class_id': 0, 'detection_id': '04fd2e4e-4cf3-41f5-a46f-e102caf98bbe', 'image_path': '../data/00:31:39.171_raw.jpg', 'prediction_type': 'ObjectDetectionModel'}]
[{'x': 183.0, 'y': 211.0, 'width': 118.0, 'height': 66.0, 'confidence': 0.891653299331665, 'class': 'fish1', 'class_id': 0, 'detection_id': '8928274e-54ee-48f2-9609-5f4c70acb711', 'image_path': '../data/00:31:52.171_raw.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 186.5, 'y': 64.0, 'width': 53.0, 'height': 36.0, 'confidence': 0.8373435735702515, 'class': 'fish1', 'class_id': 0, 'detection_id': '1a9df625-4067-490e-8760-c94d270ede70', 'image_path': '../data/00:31:52.171_raw.jpg', 'prediction_type': 'ObjectDetectionModel'}]
[{'x': 205.0, 'y': 95.0, 'width': 44.0, 'height': 38.0, 'confidence': 0.8505633473396301, 'class': 'fish1', 'class_id': 0, 'detection_id': '7a939d7a-f8d8-449e-8150-e20110535525', 'image_path'

In [5]:
for file in filenames_raw:
     predict = model.predict(os.path.join(data_dir, file), confidence=50, overlap=50).save(f"prediction_{file}")